In [1]:
import time
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')
from util import *

In [2]:
data = pd.read_csv('../../input/data.csv')

In [3]:
# 数据集1：base_info.csv
# 包含数据集7和8中涉及到的所有企业的基本信息，每一行代表一个企业的基本数据，每一行有33列，其中id列为企业唯一标识，列之间采用“,”分隔符分割。
# 数据格式如下：
# [id:企业唯一标识, oplocdistrict:行政区划代码, industryphy:行业类别代码, industryco:行业细类代码, dom:经营地址, opscope:经营范围, enttype:企业类型, enttypeitem:企业类型小类, opfrom:经营期限起, opto:经营期限止, state:状态, orgid:机构标识, jobid:职位标识, adbusign:是否广告经营, townsign:是否城镇, regtype:主题登记类型, empnum:从业人数, compform:组织形式, parnum:合伙人数, exenum:执行人数, opform:经营方式, ptbusscope:兼营范围, venind:风险行业, enttypeminu:企业类型细类, midpreindcode:中西部优势产业代码, protype:项目类型, oploc:经营场所, regcap:注册资本（金）, reccap:实缴资本, forreccap:实缴资本（外方）, forregcap:注册资本（外方）, congro:投资总额, enttypegb:企业（机构）类型]

base_info = pd.read_csv('../../input/train/base_info.csv')
print(base_info.shape)
base_info.info()

(24865, 33)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24865 entries, 0 to 24864
Data columns (total 33 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             24865 non-null  object 
 1   oplocdistrict  24865 non-null  int64  
 2   industryphy    24865 non-null  object 
 3   industryco     24864 non-null  float64
 4   dom            24865 non-null  object 
 5   opscope        24865 non-null  object 
 6   enttype        24865 non-null  int64  
 7   enttypeitem    16651 non-null  float64
 8   opfrom         24865 non-null  object 
 9   opto           8825 non-null   object 
 10  state          24865 non-null  int64  
 11  orgid          24865 non-null  int64  
 12  jobid          24865 non-null  int64  
 13  adbusign       24865 non-null  int64  
 14  townsign       24865 non-null  int64  
 15  regtype        24865 non-null  int64  
 16  empnum         19615 non-null  float64
 17  compform       10631 non-null  float64

In [4]:
base_info.head()

,id,oplocdistrict,industryphy,industryco,dom,opscope,enttype,enttypeitem,opfrom,opto,...,enttypeminu,midpreindcode,protype,oploc,regcap,reccap,forreccap,forregcap,congro,enttypegb
0,47645761dc56bb8c5fae00114b768b5d9b6e917c3aec07c4,340223,M,7513.0,31487d8f256f16bd6244b7251be2ebb24d1db51663c654...,纳米新材料、机械设备、五金配件加工、销售及技术推广服务，道路货物运输。（依法须经批准的项目，...,1100,1150.0,2019-07-11 00:00:00,NaN,...,1151.0,NaN,NaN,2367b4cac96d8598,50.0,NaN,NaN,NaN,NaN,1151
1,9c7fa510616a683058ce97d0bc768a621cd85ab1e87da2a3,340222,O,8090.0,31487d8f256f16bd6244b7251be2ebb27b17bdfd95c8f3...,健身服务。（依法须经批准的项目，经相关部门批准后方可开展经营活动）,9600,NaN,2017-09-06,NaN,...,NaN,NaN,NaN,31487d8f256f16bd6244b7251be2ebb27b17bdfd95c8f3...,10.0,NaN,NaN,NaN,NaN,9600
2,59b38c56de3836838082cfcb1a298951abfe15e6940c49ba,340202,R,9053.0,31487d8f256f16bd6244b7251be2ebb2ae36cd652943e8...,文化娱乐经纪人服务；境内文艺活动组织与策划；文化艺术交流活动组织策划；演出经纪；其他文化艺术...,1100,1150.0,2020-09-14 14:46:30,NaN,...,1151.0,NaN,NaN,2367b4cac96d8598,100.0,NaN,NaN,NaN,NaN,1151
3,e9f7b28ec10e047000d16ab79e1b5e6da434a1697cce7818,340221,L,7212.0,746df9aaed8578571760c563abe882c8ba25209fc6d5db...,投资管理及咨询(证券、期货除外)；企业管理。（依法须经批准的项目，经相关部门批准后方可开展经...,4500,4540.0,2015-09-30,NaN,...,NaN,NaN,NaN,2367b4cac96d8598,10.0,NaN,NaN,NaN,NaN,4540
4,f000950527a6feb63ee1ce82bb22ddd1ab8b8fdffa3b91fb,340202,R,8810.0,31487d8f256f16bd6244b7251be2ebb2ae36cd652943e8...,境内文化艺术交流活动策划；企业形象策划；礼仪庆典服务；翻译服务；专利代理；广告设计、制作、代...,1100,1130.0,2017-12-01,2067-11-30,...,NaN,NaN,NaN,2367b4cac96d8598,100.0,NaN,NaN,NaN,NaN,1130


In [5]:
def identify_missing(df, missing_threshold):
    """
    缺失率
    @param df:
    @param missing_threshold:
    @return:
    """
    missing_rate = df.isnull().sum() / len(df)
    missing_rate = missing_rate.sort_values(ascending=False)
    print(missing_rate)
    to_drop = missing_rate[missing_rate > missing_threshold].index.to_list()
    print('{} features with greater than {} missing values.\n'.format(len(to_drop), missing_threshold))
    return to_drop

In [6]:
to_drop = identify_missing(base_info, missing_threshold=0.9)
base_info.drop(to_drop, axis=1, inplace=True)
to_drop

midpreindcode    1.000000
ptbusscope       1.000000
protype          0.998633
forreccap        0.990871
congro           0.989986
forregcap        0.989946
exenum           0.944581
parnum           0.905932
reccap           0.715102
enttypeminu      0.707621
venind           0.660688
opto             0.645083
opform           0.638045
compform         0.572451
enttypeitem      0.330344
empnum           0.211140
regcap           0.007681
industryco       0.000040
opfrom           0.000000
oplocdistrict    0.000000
industryphy      0.000000
dom              0.000000
opscope          0.000000
enttype          0.000000
enttypegb        0.000000
state            0.000000
orgid            0.000000
jobid            0.000000
adbusign         0.000000
townsign         0.000000
regtype          0.000000
oploc            0.000000
id               0.000000
dtype: float64
8 features with greater than 0.9 missing values.



['midpreindcode',
 'ptbusscope',
 'protype',
 'forreccap',
 'congro',
 'forregcap',
 'exenum',
 'parnum']

In [7]:
base_info['opform'] = base_info['opform'].replace('01', '01-以个人财产出资').replace('02', '02-以家庭共有财产作为个人出资')

In [8]:
cat_cols = ['oplocdistrict', 'industryphy', 'industryco', 'enttype', 'enttypeitem', 'state', 'orgid', 'jobid', 'regtype', 'opform', 'venind', 'enttypeminu', 'oploc', 'enttypegb']
two_values = ['adbusign', 'townsign', 'compform', 'protype']
num_cols = ['empnum', 'parnum', 'exenum', 'regcap', 'reccap', 'forreccap', 'forregcap', 'congro']
many_cols = ['dom', 'opscope']
dt_cols = ['opfrom', 'opto']

cat_cols = [i for i in cat_cols if i not in to_drop]
two_values = [i for i in two_values if i not in to_drop]
num_cols = [i for i in num_cols if i not in to_drop]

In [9]:
base_info.to_csv('../../input/base_info.csv', index=False)

In [10]:
# dom, opscope, opfrom, opto
# 'dom', 'opscope' 取值太多
# base_info.drop(['dom', 'opscope', 'opfrom', 'opto'], axis=1, inplace=True)

In [11]:
# base_info.drop(['compform', 'regtype', 'state', 'adbusign', 'oploc', 'opform', 'venind'], axis=1, inplace=True)

In [12]:
for i in tqdm(cat_cols + many_cols):
    le = LabelEncoder()
    base_info[i] = le.fit_transform(base_info[i].astype(str))

    
for i in tqdm(two_values):
    le = LabelEncoder()
    base_info[i] = le.fit_transform(base_info[i].astype(str))

100%|███████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 107.38it/s]


In [13]:
data = data.merge(base_info, how='left', on='id')

In [14]:
train = data[data['label'].notnull()]
test = data[data['label'].isnull()]
sub = test[['id']]
# train.shape, test.shape

used_cols = [i for i in train.columns if i not in ['id', 'label', 'opfrom', 'opto']]
y = train['label']
train = train[used_cols]
test = test[used_cols]

X_train, X_valid, y_train, y_valid = train_test_split(train, y, test_size=0.25, random_state=2020)

cols = X_train.columns
useful_dict, useless_dict, useful_cols, useless_cols = auc_select(X_train, y_train, X_valid, y_valid, cols, threshold=0.52)
print('AUC useless_cols: \n', useless_cols)

dtrain = lgb.Dataset(X_train, y_train)
dvalid = lgb.Dataset(X_valid, y_valid, reference=dtrain)

params = {
    'objective': 'binary',
    'boosting': 'gbdt',
    'metric': 'auc',
#     'metric': 'None',  # 用自定义评估函数是将metric设置为'None'
    'learning_rate': 0.1,
    'num_leaves': 31,
    'lambda_l1': 0,
    'lambda_l2': 1,
    'num_threads': 23,
    'min_data_in_leaf': 20,
    'first_metric_only': True,
    'is_unbalance': True,
    'max_depth': -1,
    'seed': 2020
}

valid_model = lgb.train(
    params,
    dtrain,
    valid_sets=[dtrain, dvalid],
    early_stopping_rounds=50,
    verbose_eval=300 
)

oplocdistrict
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[8]	training's auc: 0.800172	valid_0's auc: 0.784394
Evaluated only: auc
**********
0.784394131780055
industryphy
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[33]	training's auc: 0.977366	valid_0's auc: 0.967944
Evaluated only: auc
**********
0.9679441641137959
industryco
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[104]	training's auc: 0.986598	valid_0's auc: 0.975303
Evaluated only: auc
**********
0.9753030403156906
dom
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[17]	training's auc: 0.879273	valid_0's auc: 0.820535
Evaluated only: auc
**********
0.820535347623116
opscope
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[19]	training's auc: 0.954267	valid_0's auc: 0.92122
Eval

In [15]:
importance = valid_model.feature_importance(importance_type='gain')
feature_name = valid_model.feature_name()

df_importance = pd.DataFrame({
    'feature_name': feature_name,
    'importance': importance
}).sort_values(by='importance', ascending=False)
df_importance['normalized_importance'] = df_importance['importance'] / df_importance['importance'].sum()
df_importance['cumulative_importance'] = np.cumsum(df_importance['normalized_importance'])
df_importance

,feature_name,importance,normalized_importance,cumulative_importance
1,industryphy,114776.834858,0.882121,0.882121
19,regcap,4067.029875,0.031257,0.913378
20,reccap,1727.581344,0.013277,0.926655
3,dom,1581.706230,0.012156,0.938811
2,industryco,1334.548596,0.010257,0.949068
4,opscope,1326.564802,0.010195,0.959263
21,enttypegb,1024.956897,0.007877,0.967141
9,jobid,863.183075,0.006634,0.973775
6,enttypeitem,744.967502,0.005725,0.979500
13,empnum,700.865225,0.005387,0.984887


In [16]:
record_low_importance = df_importance[df_importance['importance'] == 0.0]
to_drop = list(record_low_importance['feature_name'])
print(to_drop)

# base_info.drop(to_drop, axis=1, inplace=True)
base_info.to_csv('../../input/fe_base_info.csv', index=False)

['adbusign', 'regtype', 'opform', 'venind', 'oploc', 'state']


In [17]:
# y_test_pred = np.where(valid_model.predict(test) > 0.5, 1, 0)

# print('Test mean label: ', np.mean(y_test_pred))
# sub['score'] = y_test_pred
# sub.to_csv('../../sub/baseline_{}.csv'.format(time.strftime('%Y%m%d')), index=False)